In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px

In [7]:
# https://www.data.go.kr/data/15049591/fileData.do <= 표준산업분류코드 데이터
def std_convert(str):
    return f"0{str}" if len(str) == 1 else str
STD_TOBZ_M_df = pd.read_csv("dataset/고용노동부_표준산업분류코드.csv")
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].astype(int).astype(str)
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].apply(lambda x : std_convert(x))
STD_TOBZ_M_df['고용업종코드(소분류)'] = STD_TOBZ_M_df['고용업종코드(소분류)'].astype(str)

STD_TOBZ_M_dic = dict(zip(STD_TOBZ_M_df['고용업종코드(중분류)'], STD_TOBZ_M_df['고용업종명(중분류)'])) # 표준산업분류 - 중분류 dict
STD_TOBZ_S_dic = dict(zip(STD_TOBZ_M_df['고용업종코드(소분류)'], STD_TOBZ_M_df['고용업종명(소분류)'])) # 표준산업분류 - 소분류 dict

In [6]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

In [17]:
import dash
import dash_deck
import dash_html_components as html

C:\Users\user\AppData\Local\Temp\ipykernel_11396\4139716189.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [20]:
deck_component = dash_deck.DeckGL(r.to_json(), id="deck-gl")

app = dash.Dash(__name__)
app.layout = html.Div(deck_component)

In [ ]:
html_str = r.to_html()
# HTML을 JSON으로 변환
json_data = {
    "pydeck": html_str
}

# JSON 파일로 저장
with open("pydeck_visualization.json", "w") as json_file:
    json.dump(json_data, json_file)

In [12]:
# km data preprocessing
def exc_age(age):
    age = age.split(".")[-1].split("대")[0]
    try:
        return "60+" if int(age) >= 60 else age
    except:
        return age
km_data = pd.read_csv("dataset/국민카드.csv", encoding='cp949')
km_data = km_data[~km_data['표준산업세세분류코드'].isna()]
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].astype(int).astype(str)
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:3])
km_data['STD_TOBZ_M_C'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:2])
km_data['기준년월'] = pd.to_datetime(km_data['기준년월'], format='%Y%m')
km_data['연령대'] = km_data['연령대'].apply(exc_age)
km_data['company'] = '국민'
km_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'code_s_name', 'code_s', 'cost', 'case', 'client', 'code_m', 'company']
km_data = km_data[['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']]
km_data['case'] = km_data['case'].astype(int)

# sh data preprocessing
city_l_dic = {'전남':'전라남도', '경기':'경기도', '서울':'서울특별시'}
sex_dic = {'F':'여성', 'M':'남성'}
sh_data = pd.read_csv("dataset/신한카드.csv")
sh_data['MGPO_NM_HOM'] = sh_data['MGPO_NM_HOM'].map(city_l_dic)
sh_data['SEX_C'] = sh_data['SEX_C'].map(sex_dic)
sh_data['SL_CRI_YM'] = pd.to_datetime(sh_data['SL_CRI_YM'], format='%Y%m')
sh_data['STD_TOBZ_M_C'] = sh_data['STD_TOBZ_M_C'].astype(str)
sh_data['company'] = '신한'
sh_data = sh_data[['SL_CRI_YM', 'MGPO_NM_HOM', 'SGG_NM_HOM', 'NASD_NM_HOM', 'SEX_C', 'AGE_G5_C', 'company', 'STD_TOBZ_M_C', 'STD_TOBZ_L_C','AMT', 'CNT']]
sh_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
sh_data['case'] = sh_data['case'].apply(lambda x : int(x) if x.isdigit() else 0)

# ss data preprocessing
def exc_age(age):
    return "60+" if int(age) >= 60 else age      
ss_data = pd.read_csv("dataset/삼성카드.csv")
ss_data['표준업종분류코드_중분류'] = ss_data['표준업종분류코드_중분류'].astype(str)
ss_data['거주지_광역시도'] = ss_data['거주지_광역시도'].map(city_l_dic)
ss_data['성별'] = ss_data['성별'].map(sex_dic)
ss_data['이용기준년월'] = pd.to_datetime(ss_data['이용기준년월'], format='%Y%m')
ss_data['연령대'] = ss_data['연령대'].apply(exc_age)
ss_data['company'] = '삼성'
ss_data = ss_data[['이용기준년월', '거주지_광역시도', '거주지_시군구', '거주지_행정동', '성별', '연령대', 'company', '표준업종분류코드_중분류', '표준업종분류코드_소분류','이용금액', '이용건수']]
ss_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
ss_data['case'] = ss_data['case'].astype(int)

# data concat
con_df = pd.concat([km_data, sh_data, ss_data]).reset_index(drop=True) # 3사 데이터 통합
con_df = con_df.drop_duplicates()
jg_df = con_df[con_df['city_m']=='중구'].reset_index(drop=True) # 중구 data만 filtering
ddc_df = con_df[con_df['city_m']=='동두천시'].reset_index(drop=True) # 동두천시 data만 filtering
nj_df = con_df[con_df['city_m']=='나주시'].reset_index(drop=True) # 나주시 data만 filtering

In [ ]:
jg_count_dic = dict(zip(jg_df['code_m'].value_counts().index,jg_df['code_m'].value_counts().values))
jg_std = pd.DataFrame({'표준산업분류코드':sorted(jg_df['code_m'].unique())})
jg_std['표준산업분류(중)'] = jg_std['표준산업분류코드'].map(STD_TOBZ_M_dic)
jg_std['데이터건수'] = jg_std['표준산업분류코드'].map(jg_count_dic)
jg_std = jg_std.sort_values(by='데이터건수').reset_index(drop=True)
jg_std